In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pathlib
import os
import copy
import datetime
import time
# sklearn?

import seaborn as sns
sns.set_style()

In [2]:
data_origin = pd.read_csv("Data/subset-3-sous-ensemble-3.csv", encoding='latin1')
print(data_origin.shape)
print(data_origin.columns)

(1405579, 35)
Index(['LEVEL1ID', 'LEVEL2ID', 'LEVEL3ID', 'LEVEL4ID', 'LEVEL5ID', 'SURVEYR',
       'BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'DEMCODE', 'QUESTION', 'TITLE_E',
       'TITLE_F', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'DEPT_F',
       'INDICATORID', 'INDICATORENG', 'INDICATORFRA', 'SUBINDICATORID',
       'SUBINDICATORENG', 'SUBINDICATORFRA'],
      dtype='object')


In [3]:
data = data_origin.copy()

# drop columns with only 1 value
# empty = []
# for col in data.columns:
#     uni = data[col].unique()
#     if len(uni)<2:
#         print(col, uni)
#         data.drop(columns=[col], inplace=True)
try: data.drop(columns=['LEVEL2ID', 'LEVEL3ID', 'LEVEL4ID', 'LEVEL5ID'], inplace=True)
except: pass

# drop useless columns
# DEMCODE = BYCOND, DESCRIP_E, DESCRIP_F
# QUESTION = TITLE_E, TITLE_F
# INDICATORID = INDICATORENG, INDICATORFRA
# SUBINDICATORID = SUBINDICATORENG, SUBINDICATORFRA
# LEVEL1ID = DEPT_E, DEPT_F
# SCORE5 = X*SCORE100 + Y
useless = ['BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'TITLE_E', 'TITLE_F', 'SUBINDICATORENG', 'SUBINDICATORFRA', 'DEPT_E', 'DEPT_F', 'INDICATORENG', 'INDICATORFRA', 'SCORE100']
try: data.drop(columns=useless, inplace=True)
except: pass

# drop empty lines
data.drop(data[data['ANSWER1'] == ' '].index, inplace=True)

# change strings to numbers
traduction_question = {ques: i+101 for i, ques in enumerate(data['QUESTION'].unique())}
data['QUESTION'] = data['QUESTION'].map(lambda x: traduction_question[x])

data = data.astype({'SCORE5': 'float'}, copy=False)
data = data.astype({col: 'int' for col in ['ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
    'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE',
    'ANSCOUNT']}, copy=False)

# add unique ids to each participant
data['ID'] = data['LEVEL1ID']*1000 + (data['SURVEYR']-2018)*100 + data['DEMCODE']-2011

print(data.shape)
print(data.columns)

(583123, 20)
Index(['LEVEL1ID', 'SURVEYR', 'DEMCODE', 'QUESTION', 'ANSWER1', 'ANSWER2',
       'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
       'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY',
       'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE', 'SCORE5', 'ANSCOUNT',
       'INDICATORID', 'SUBINDICATORID', 'ID'],
      dtype='object')


In [4]:
for col in data.columns:
    print(col, '\t', len(data[col].unique()), '\t', type(data[col][0]), '\t', data[col][0])

LEVEL1ID 	 68 	 <class 'numpy.int64'> 	 0
SURVEYR 	 3 	 <class 'numpy.int64'> 	 2020
DEMCODE 	 59 	 <class 'numpy.int64'> 	 2011
QUESTION 	 216 	 <class 'numpy.int64'> 	 101
ANSWER1 	 101 	 <class 'numpy.int64'> 	 35
ANSWER2 	 101 	 <class 'numpy.int64'> 	 45
ANSWER3 	 87 	 <class 'numpy.int64'> 	 6
ANSWER4 	 71 	 <class 'numpy.int64'> 	 11
ANSWER5 	 82 	 <class 'numpy.int64'> 	 3
ANSWER6 	 74 	 <class 'numpy.int64'> 	 0
ANSWER7 	 65 	 <class 'numpy.int64'> 	 0
MOST_POSITIVE_OR_LEAST_NEGATIVE 	 102 	 <class 'numpy.int64'> 	 81
NEUTRAL_OR_MIDDLE_CATEGORY 	 86 	 <class 'numpy.int64'> 	 6
MOST_NEGATIVE_OR_LEAST_POSITIVE 	 98 	 <class 'numpy.int64'> 	 14
AGREE 	 102 	 <class 'numpy.int64'> 	 81
SCORE5 	 366 	 <class 'numpy.float64'> 	 3.99
ANSCOUNT 	 15718 	 <class 'numpy.int64'> 	 73497
INDICATORID 	 6 	 <class 'numpy.int64'> 	 4
SUBINDICATORID 	 23 	 <class 'numpy.int64'> 	 14
ID 	 4992 	 <class 'numpy.int64'> 	 200


In [11]:
data_questions = data.drop(columns = ['SURVEYR', 'DEMCODE', 'LEVEL1ID'])
list_id = data_questions['ID'].unique()
set_id = set(list_id)
data_agg = pd.DataFrame({'ID' : list_id})

for i in range(101, 317):
    new_question = data_questions[data_questions['QUESTION']==i].drop(columns = ['QUESTION'])
    missing_id = list(set_id-set(new_question['ID']))
        
    new_df = pd.DataFrame({name: ([0]*len(missing_id) if index<len(new_question.columns)-1 else missing_id) for index, name in enumerate(new_question.columns)})
    new_question = new_question.append(new_df, ignore_index = True)
            
    new_question.rename(columns=lambda x: x + "_" + str(i) if x!='ID' else x, inplace=True)
    data_agg = pd.merge(data_agg, new_question, on = 'ID')

data_agg['LEVEL1ID'] = data_agg['ID'] // 1000
data_agg['SURVEYR'] = (data_agg['ID'] // 100) % 10 + 2018
data_agg['DEMCODE'] = data_agg['ID'] % 100 + 2011

print(data_agg.shape)
print(data_agg.columns)

(4992, 3244)
Index(['ID', 'ANSWER1_101', 'ANSWER2_101', 'ANSWER3_101', 'ANSWER4_101',
       'ANSWER5_101', 'ANSWER6_101', 'ANSWER7_101',
       'MOST_POSITIVE_OR_LEAST_NEGATIVE_101', 'NEUTRAL_OR_MIDDLE_CATEGORY_101',
       ...
       'NEUTRAL_OR_MIDDLE_CATEGORY_316', 'MOST_NEGATIVE_OR_LEAST_POSITIVE_316',
       'AGREE_316', 'SCORE5_316', 'ANSCOUNT_316', 'INDICATORID_316',
       'SUBINDICATORID_316', 'LEVEL1ID', 'SURVEYR', 'DEMCODE'],
      dtype='object', length=3244)


/tmp/ipykernel_1166/1844093436.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_agg['LEVEL1ID'] = data_agg['ID'] // 1000
/tmp/ipykernel_1166/1844093436.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_agg['SURVEYR'] = (data_agg['ID'] // 100) % 10 + 2018
/tmp/ipykernel_1166/1844093436.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To g